In [1]:
# from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")

In [2]:
import pickle
# with open('yelp_dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [3]:
with open('yelp_dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

/home/sriteja/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [5]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [6]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


## Preprocess

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sriteja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sriteja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import sys  
!{sys.executable} -m pip install contractions
import contractions

In [9]:
# import re  # Import the regular expressions module

# # use tokenizer to remove punctuation

# def tokenize_text(text):
#     # Replace "!" with "exm" using regular expressions
#     text = re.sub(r'!', ' exm', text)
    
#     expanded_words = []
#     for word in text.split():
#         # using contractions.fix to expand the shortened words
#         expanded_words.append(contractions.fix(word))   
    
#     expanded_text = ' '.join(expanded_words)
#     # print(expanded_text)
    
#     text = expanded_text    
    
#     # replace a-b with a and b
#     text = text.replace('-', ' ')
    
#     tokens = nltk.word_tokenize(text)
    
#     # Add an extra occurrence for all-uppercase words with more than one letter
#     # temp = [word if (len(word) > 1 and word.isupper()) else None for word in tokens]
#     temp = []
#     for word in tokens:
#         if len(word) > 1 and word.isupper():
#             temp.append(word)
#     tokens.extend(x for x in temp if x)
    
#     # convert to lower case
#     tokens = [w.lower() for w in tokens]
#     # dr. = dr and st. = st and so on
#     tokens = [w.replace('.', '') for w in tokens]
    
#     # remove punctuation
#     # tokens = [word for word in tokens if word.isalpha()]
    
#     # Remove stop words
#     stop_words = set(stopwords.words("english"))
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    
#     return tokens

# # print(train_data[4])
# # tokenize_text(train_data[4])


In [10]:
# tokens = []
# print(len(train_data))

# for i in range(len(train_data)):
#     tokens.append(tokenize_text(train_data[i]))
#     if i % 1000 == 0:
#         print(i)

# # save to pkl file
# with open('yelp_train_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens, file)

In [11]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('yelp_train_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 10

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('yelp_train_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [12]:
# tokens_test = []
# print(len(test_data))

# for i in range(len(test_data)):
#     tokens_test.append(tokenize_text(test_data[i]))
#     if i % 1000 == 0:
#         print(i)

# with open('yelp_test_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens_test, file)

In [13]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('yelp_test_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 10

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('yelp_test_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [14]:
import numpy as np

train_tokens = []
with open('yelp_train_tokens_filtered.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

test_tokens = []
with open('yelp_test_tokens_filtered.pkl', 'rb') as file:
    test_tokens = pickle.load(file)
    
print("Loaded tokens")



Loaded tokens


In [15]:
# in train data replace every 1000th word with UNK randomly

import random

for i in range(len(train_tokens)):
    for j in range(len(train_tokens[i])):
        if random.randint(1, 1000) == 1:
            train_tokens[i][j] = 'UNK'
        

In [16]:
# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for tokens in train_tokens:
    vocab.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict = {}
for i, word in enumerate(vocab):
    vocab_dict[word] = i

print(len(vocab_dict))
# Initialize BoW matrices for training and testing data


45287


In [17]:
reduced_train_tokens = train_tokens[:20000]

# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab_reduced = set()
for tokens in reduced_train_tokens:
    vocab_reduced.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict_reduced = {}
for i, word in enumerate(vocab_reduced):
    vocab_dict_reduced[word] = i

print(len(vocab_dict_reduced))

train_bow = np.zeros((len(reduced_train_tokens), len(vocab_reduced)))

print("bow train")
# Convert text to BoW vectors
for i, tokens in enumerate(reduced_train_tokens):
    if i % 5000 == 0:
        print(i)
    for token in tokens:
        train_bow[i][vocab_dict_reduced[token]] += 1




27977
bow train
0
5000
10000
15000


In [18]:
reduced_test_tokens = test_tokens[:5000]
test_bow = np.zeros((len(reduced_test_tokens), len(vocab_reduced)))

print("bow test")

for i, tokens in enumerate(reduced_test_tokens):
    if i % 1000 == 0:
        print(i)
    for token in tokens:
        if token in vocab_dict_reduced:
            test_bow[i][vocab_dict_reduced[token]] += 1
        else:    # if there is an unknown word, add it to the UNK column 
            test_bow[i][vocab_dict_reduced['UNK']] += 1

bow test
0
1000
2000
3000
4000


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Logistic Regression classifier
classifier = LogisticRegression(max_iter=200)

# Fit the classifier to the training data with a progress bar
# with tqdm(total=len(train_bow[:20000])) as pbar:
classifier.fit(train_bow[:20000], train_labels[:20000])
    # pbar.update(len(train_bow[:20000]))

# Predict the labels of the test data with a progress bar
# with tqdm(total=len(test_bow[:5000])) as pbar:
y_pred = classifier.predict(test_bow[:5000])
    # pbar.update(len(test_bow[:5000]))

# Calculate accuracy and other classification metrics
print('Accuracy: {}'.format(accuracy_score(test_labels[:5000], y_pred)))
print(classification_report(test_labels[:5000], y_pred))

# save the classifier
with open('yelp_classifier_log_reg_06_200itr.pkl', 'wb') as file:
    pickle.dump(classifier, file)

/home/sriteja/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.497
              precision    recall  f1-score   support

           0       0.68      0.66      0.67      1141
           1       0.41      0.42      0.41      1009
           2       0.40      0.40      0.40      1003
           3       0.43      0.40      0.42       986
           4       0.56      0.59      0.57       861

    accuracy                           0.50      5000
   macro avg       0.49      0.49      0.49      5000
weighted avg       0.50      0.50      0.50      5000

